In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
import os, cv2, json,sys
from PIL import Image
# from keras.models import load_model

from sklearn.model_selection import train_test_split
import tensorflow.keras.applications.efficientnet as efn

from tensorflow.keras import backend as K
# ignoring warnings
import warnings
warnings.simplefilter("ignore")

seed = 2021

tf.random.set_seed(seed)
np.random.seed(seed)
os.environ['PYTHONSEED'] =str(seed)

In [ ]:
# For easy acces to files
WORK_DIR = "../input/cassava-leaf-disease-classification/"
os.listdir(WORK_DIR)

In [ ]:
with open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as file:
    labels = json.load(file)
    
labels

In [ ]:
CLASSES = ['0', '1', '2', '3', '4']

In [ ]:
data = pd.read_csv(WORK_DIR + "train.csv")

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
#change for the ImageDatagen and flow_from_dataframe
data.label = data.label.astype("str")

In [ ]:
data.dtypes

In [ ]:
data.shape[0]

In [ ]:
data.label.value_counts()

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 256
CHANNELS = 3
img_size = (IMG_SIZE, IMG_SIZE)

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data.sample(9).reset_index(drop=True)

for i in range(8):
    plt.subplot(2,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Cassava Bacterial Blight (CBB)**

In [ ]:
labels.get("0")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="0"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Cassava Brown Streak Disease (CBSD)**

In [ ]:
labels.get("1")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="1"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Cassava Green Mottle (CGM)**

In [ ]:
labels.get("2")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="2"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Cassava Mosaic Disease (CMD)**

In [ ]:
labels.get("3")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="3"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Healthy**

In [ ]:
labels.get("4")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="4"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

Split data

In [ ]:
train, valid = train_test_split(data, test_size = 0.1, \
                              random_state = seed, stratify = data['label'])

train = train.reset_index(drop= True)
valid = valid.reset_index(drop= True)

In [ ]:
print(train.shape, valid.shape)

In [ ]:
train_generator = ImageDataGenerator(
                                    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    brightness_range=[0.1,0.9],
                                    shear_range=25,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode= 'nearest'
) \
        .flow_from_dataframe(
                            train,
                            directory = WORK_DIR + "train_images",
                            x_col = "image_id",
                            y_col = "label",
                            target_size = img_size,
                            class_mode = "categorical",
                            interpolation = "nearest",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            seed = seed,    
)


In [ ]:
valid_generator = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input
) \
        .flow_from_dataframe(
                            valid,
                            directory = WORK_DIR + "train_images",
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (260,260),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            interpolation = "nearest",
                            shuffle = False,
                            )

In [ ]:
valid_generator.class_indices

In [ ]:

class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=3e-5, 
    decay_steps=10000, 
    decay_rate=0.9)

In [ ]:
weights_path = '../input/keras-efficientnetb3-noisy-student/noisy_student_efficientnet_b4.h5'

def build_model():
    
    input_img = layers.Input(shape=(IMG_SIZE,IMG_SIZE,CHANNELS), name='input_image')
    
#     img_bn = layers.BatchNormalization()(input_img)
    
    x = efn.EfficientNetB4(include_top = False,
                        weights = weights_path,
                        drop_connect_rate=0.5
                        )(input_img)
    
    avg_pool = layers.GlobalAveragePooling2D()(x)
    
    
    h1 = layers.Dense(1792, activation='relu', name='h1', \
                     bias_regularizer = tf.keras.regularizers.L1L2(l1 = 0.01, l2 = 0.001))(avg_pool)
      
    h2 = layers.add([avg_pool, h1])
    h3 = layers.add([h2, layers.Dense(1792, activation='relu',name='h3')(h2)])

    drop = layers.Dropout(rate= 0.25, name ='drop')(h3)

    predictions = layers.Dense(len(CLASSES), activation='softmax',kernel_regularizer= \
                               tf.keras.regularizers.L2(0.01))(drop)
    
    model = models.Model(inputs = input_img , outputs = predictions)
    
    return model

In [ ]:
print(build_model().summary())

In [ ]:
def symmetric_cross_entropy(alpha, beta):
    def loss(y_true, y_pred):
        y_true_1 = y_true
        y_pred_1 = y_pred

        y_true_2 = y_true
        y_pred_2 = y_pred

        y_pred_1 = tf.clip_by_value(y_pred_1, 1e-7, 1.0)
        y_true_2 = tf.clip_by_value(y_true_2, 1e-4, 1.0)

        return alpha*tf.reduce_mean(-tf.reduce_sum(y_true_1 * tf.math.log(y_pred_1), axis = -1)) + beta*tf.reduce_mean(-tf.reduce_sum(y_pred_2 * tf.math.log(y_true_2), axis = -1))
    return loss

In [ ]:
loss_fn = symmetric_cross_entropy(alpha=1.0,beta=1.0)

In [ ]:
# with strategy.scope():
model = build_model()
    
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate= lr_schedule, epsilon=0.001),

loss= tf.keras.losses.CategoricalCrossentropy(
        from_logits = False,
        label_smoothing=0.0001,
        name='categorical_crossentropy'
    ),  
metrics=['accuracy'])

In [ ]:
from tensorflow.keras import utils

utils.plot_model(model)

In [ ]:
model_check = ModelCheckpoint(
                            "./best_model.h5",
                            monitor = "val_loss",
                            verbose = 1,
                            save_best_only = True,
                            save_weights_only = True,
                            mode = "min")

early_stop= EarlyStopping(
                        monitor = "val_loss",
                        min_delta=0.001,
                        patience=5,
                        verbose=1,
                        mode="min",
                        restore_best_weights=True)

In [ ]:
EPOCHS = 20
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VAL = valid_generator.n // valid_generator.batch_size

In [ ]:
history = model.fit_generator(train_generator,
                                epochs = EPOCHS,
                                validation_data = valid_generator,
                                steps_per_epoch = STEP_SIZE_TRAIN,
                                validation_steps = STEP_SIZE_VAL,
                                callbacks = [model_check,early_stop])

# model.save('leaf_disease_model.h5')

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['accuracy'], 'b*-', label="train_acc")
plt.plot(history.history['val_accuracy'], 'r*-', label="val_acc")
plt.grid()
plt.title("train_acc vs val_acc")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['loss'], 'b*-', label="train_loss")
plt.plot(history.history['val_loss'], 'r*-', label="val_loss")
plt.grid()
plt.title("train_loss - val_loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
# model = load_model('./best_model.h5')

In [ ]:
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = Image.open(TEST_DIR + image)
    img = img.resize(img_size)
    img = np.expand_dims(img, axis=0)
    predictions.extend(model.predict(img).argmax(axis = 1))

In [ ]:
# model.save('model.h5')

In [ ]:
# model.save('model_weights.h5')

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
display(sub)
sub.to_csv('submission.csv', index = False)